In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
import time
import concurrent.futures  # 병렬 처리

In [5]:
# USER HEADER
headers = [{'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}]

def call_site(page) :
    url = f'https://gall.dcinside.com/mgallery/board/lists/?id=dfip&page={page}' # 던전앤파이터 갤러리, 던파아이피 갤러리
    try :
        response = requests.get(url, headers=headers[0])
        print(f'페이지 {page}번 진행중입니다.')
    except :
        print('error')
        
    if response.status_code != 200:
        print(f'오류: HTTP 상태 코드 {response.status_code}')
        return None
    
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

def call_content(page ,title_num) :
    url = f'https://gall.dcinside.com/mgallery/board/view/?id=dfip&no={title_num}&page={page}'
    try :
        response = requests.get(url, headers=headers[0])
    except :
        print(f'{page}페이지, {title_num}번 게시글 : error')
        
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import concurrent.futures  # 병렬 처리를 위한 라이브러리

# 기본 설정
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}
start_page = 3180
end_page = 21650
wait_time = 1
cut_off_date = '2024-12-01'
max_workers = 5  # 동시에 실행할 작업 수

# 페이지 불러오는 함수
def call_site(page):
    url = f'https://gall.dcinside.com/mgallery/board/lists/?id=dfip&page={page}'
    try:
        response = requests.get(url, headers=headers)
        print(f'페이지 {page}번 진행중입니다.')
    except:
        print('error')
        return None
        
    if response.status_code != 200:
        print(f'오류: HTTP 상태 코드 {response.status_code}')
        return None
    
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

# 게시글 내용 불러오는 함수
def call_content(page, title_num):
    url = f'https://gall.dcinside.com/mgallery/board/view/?id=dfip&no={title_num}&page={page}'
    try:
        response = requests.get(url, headers=headers)
    except:
        print(f'{page}페이지, {title_num}번 게시글 : error')
        return None
        
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

# 한 페이지 처리하는 함수
def process_page(page_num):
    # 결과를 담을 리스트
    page_results = {
        'title_num': [],
        'write_time': [],
        'nickname': [],
        'title_text': [],
        'content': []
    }
    
    # 대기
    time.sleep(wait_time)
    
    # 페이지 불러오기
    soup = call_site(page_num)
    if soup is None:
        return page_results, False
    
    try:
        # tbody 요소 확인 (수정된 부분)
        tbody = soup.find('tbody')
        if tbody is None:
            print(f"페이지 {page_num}: tbody 요소를 찾을 수 없음, 다시 시도합니다.")
            # 잠시 대기 후 재시도
            time.sleep(1)
            soup = call_site(page_num)
            if soup is None:
                return page_results, False
                
            tbody = soup.find('tbody')
            if tbody is None:
                print(f"페이지 {page_num}: tbody 요소를 두 번째 시도에서도 찾을 수 없음, 건너뜁니다.")
                return page_results, False
        
        # 데이터 로드
        article_list = tbody.find_all('tr')
        
        # 페이지 내 게시글 처리
        for i in range(1, len(article_list)):
            # html 전처리
            article = article_list[i]
            
            # 게시글 번호
            title_num = article.find("td", {"class": "gall_num"}).text
            if not title_num.isdigit():  # 없으면 넘기기
                continue
            
            # 닉네임 추출
            try:
                nickname = article.find("span", {'class': "nickname in"}).text
            except AttributeError:
                try:
                    sub_nickname = article.find("span", {'class': "nickname"})['title']
                    sub_ip = article.find("span", {'class': "ip"}).text
                    nickname = f"{sub_nickname}{sub_ip}"
                except:
                    nickname = "추출실패"
            
            # 게시글 제목
            try:
                title_text = article.find_all("a")[0].text.strip()
            except:
                title_text = "제목추출실패"
            
            # 게시글 작성 시간 추출
            try:
                write_time = article.find("td", {'class': 'gall_date'})['title']
                write_time = pd.to_datetime(write_time).strftime("%Y-%m-%d")
                
                # 날짜 체크
                if write_time < cut_off_date:
                    return page_results, True  # 중단 신호 반환
            except:
                continue
            
            # 게시글 내용 추출
            try:
                content_soup = call_content(page=page_num, title_num=title_num)
                if content_soup is None:
                    continue
                    
                content_div = content_soup.find('div', class_='write_div')
                if content_div is None:
                    continue
                    
                content = content_div.text.strip().split()
                
                # 리스트에 데이터 축적
                page_results['title_num'].append(title_num)
                page_results['title_text'].append(title_text)
                page_results['nickname'].append(nickname)
                page_results['write_time'].append(write_time)
                page_results['content'].append(content)
            except Exception as e:
                print(f"내용 추출 오류 (페이지 {page_num}, 게시글 {title_num}): {str(e)}")
                continue
    
    except Exception as e:
        print(f"페이지 {page_num} 처리 중 오류: {str(e)}")
    
    return page_results, False

# 메인 함수
def main():
    # 데이터 축적용 딕셔너리
    all_results = {
        'title_num': [],
        'write_time': [],
        'nickname': [],
        'title_text': [],
        'content': []
    }
    
    # 병렬 처리를 위한 ThreadPoolExecutor 사용
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 페이지 범위를 나누어 처리 (한 번에 너무 많은 페이지를 처리하지 않음)
        chunk_size = 50  # 한 번에 50페이지씩 처리
        
        for chunk_start in range(start_page, end_page + 1, chunk_size):
            chunk_end = min(chunk_start + chunk_size - 1, end_page)
            print(f"페이지 {chunk_start}~{chunk_end} 처리 중...")
            
            # 각 페이지에 process_page 함수 실행
            futures = {executor.submit(process_page, page_num): page_num 
                      for page_num in range(chunk_start, chunk_end + 1)}
            
            # 작업 결과 수집
            should_break = False
            for future in concurrent.futures.as_completed(futures):
                page_num = futures[future]
                
                try:
                    page_results, stop_signal = future.result()
                    
                    # 결과 합치기
                    for key in all_results:
                        all_results[key].extend(page_results[key])
                    
                    print(f"페이지 {page_num} 완료: {len(page_results['title_num'])}개의 게시글 처리됨")
                    
                    # 중단 신호 처리
                    if stop_signal:
                        should_break = True
                        break
                        
                except Exception as e:
                    print(f"페이지 {page_num} 처리 실패: {str(e)}")
            
            # 중간 결과 저장
            temp_df = pd.DataFrame(all_results)
            temp_df.to_csv(f"dnf_dcinside_content_temp_{chunk_start}-{chunk_end}.csv", index=False)
            print(f"중간 결과 저장됨: {len(temp_df)}개 게시글")
            
            # 중단 신호 확인
            if should_break:
                print("날짜 기준 도달로 크롤링 종료")
                break
    
    # 최종 데이터프레임 생성 및 저장
    final_df = pd.DataFrame(all_results)
    final_df.to_csv(f"dnf_dcinside_content_414-{end_page}.csv", index=False)
    print(f"크롤링 완료! 총 {len(final_df)}개의 게시글 수집됨")

# 프로그램 실행
if __name__ == "__main__":
    start_time = time.time()
    
    try:
        main()
    except KeyboardInterrupt:
        print("\n사용자에 의해 중단됨")
    except Exception as e:
        print(f"예기치 못한 오류: {str(e)}")
    
    elapsed_time = time.time() - start_time
    print(f"총 소요 시간: {elapsed_time:.2f}초")

페이지 3180~3229 처리 중...
페이지 3182번 진행중입니다.
페이지 3182: tbody 요소를 찾을 수 없음, 다시 시도합니다.
페이지 3180번 진행중입니다.
페이지 3180: tbody 요소를 찾을 수 없음, 다시 시도합니다.
페이지 3184번 진행중입니다.
페이지 3184: tbody 요소를 찾을 수 없음, 다시 시도합니다.
페이지 3181번 진행중입니다.
페이지 3181: tbody 요소를 찾을 수 없음, 다시 시도합니다.
페이지 3183번 진행중입니다.
페이지 3183: tbody 요소를 찾을 수 없음, 다시 시도합니다.
페이지 3183번 진행중입니다.
페이지 3183: tbody 요소를 두 번째 시도에서도 찾을 수 없음, 건너뜁니다.
페이지 3183 완료: 0개의 게시글 처리됨
페이지 3184번 진행중입니다.
페이지 3184: tbody 요소를 두 번째 시도에서도 찾을 수 없음, 건너뜁니다.
페이지 3181번 진행중입니다.
페이지 3181: tbody 요소를 두 번째 시도에서도 찾을 수 없음, 건너뜁니다.
페이지 3182번 진행중입니다.
페이지 3182: tbody 요소를 두 번째 시도에서도 찾을 수 없음, 건너뜁니다.
페이지 3184 완료: 0개의 게시글 처리됨
페이지 3181 완료: 0개의 게시글 처리됨
페이지 3182 완료: 0개의 게시글 처리됨
페이지 3180번 진행중입니다.
페이지 3180: tbody 요소를 두 번째 시도에서도 찾을 수 없음, 건너뜁니다.
페이지 3180 완료: 0개의 게시글 처리됨
페이지 3187번 진행중입니다.
페이지 3187: tbody 요소를 찾을 수 없음, 다시 시도합니다.
페이지 3189번 진행중입니다.
페이지 3189: tbody 요소를 찾을 수 없음, 다시 시도합니다.
페이지 3188번 진행중입니다.
페이지 3188: tbody 요소를 찾을 수 없음, 다시 시도합니다.
페이지 3185번 진행중입니다.
페이지 3185: tbody 요소를 찾을 수 없음, 다시 시도합니다.
페이지 3186번 진행중입니

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import concurrent.futures
import datetime

# 기본 설정
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}
start_page = 3180
end_page = 21650
wait_time = 1
cut_off_date = '2024-12-01'
max_workers = 2  # 차단 방지를 위해 동시 작업 수 감소
block_wait_time = 300  # 차단 감지 시 대기 시간(초)

# 여러 개의 User-Agent를 사용하여 차단 방지
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0'
]

# 차단 감지 플래그
is_blocked = False
block_start_time = None

def get_random_headers():
    """랜덤 User-Agent를 포함한 헤더 반환"""
    return {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/html,application/xhtml+xml,application/xml',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    }

def check_if_blocked(soup):
    """차단 여부 확인 함수"""
    # 디시인사이드의 차단 페이지 특성을 확인
    # 1. tbody가 없는 경우
    if soup.find('tbody') is None:
        return True
    
    # 2. 특정 오류 메시지가 포함된 경우
    error_texts = ['접근이 차단되었습니다', '비정상적인 접근', '일시적으로 접근이 제한되었습니다']
    page_text = soup.get_text().lower()
    for error in error_texts:
        if error.lower() in page_text:
            return True
    
    return False

def handle_blocking():
    """차단 감지 시 처리 함수"""
    global is_blocked, block_start_time
    
    current_time = datetime.datetime.now()
    if is_blocked:
        # 이미 차단 상태이면 대기 시간 확인
        elapsed_seconds = (current_time - block_start_time).total_seconds()
        if elapsed_seconds < block_wait_time:
            remaining = block_wait_time - elapsed_seconds
            print(f"차단 상태입니다. {remaining:.0f}초 더 대기 중...")
            time.sleep(min(remaining, 10))  # 최대 10초씩 대기하며 확인
            return True
        else:
            print("대기 시간이 끝났습니다. 다시 시도합니다.")
            is_blocked = False
            return False
    else:
        # 새로운 차단 감지
        print(f"차단이 감지되었습니다. {block_wait_time}초 대기합니다.")
        is_blocked = True
        block_start_time = current_time
        time.sleep(10)  # 초기 10초 대기
        return True

def call_site(page, max_retries=3):
    """페이지 불러오는 함수 (차단 감지 및 재시도 로직 포함)"""
    global is_blocked
    
    # 차단 상태 확인
    if is_blocked and handle_blocking():
        return None
    
    url = f'https://gall.dcinside.com/mgallery/board/lists/?id=dfip&page={page}'
    
    for retry in range(max_retries):
        try:
            # 무작위 대기 시간 (차단 방지)
            delay = wait_time + random.uniform(0, 0.3)
            time.sleep(delay)
            
            # 랜덤 헤더 사용
            current_headers = get_random_headers()
            response = requests.get(url, headers=current_headers, timeout=10)
            
            print(f'페이지 {page}번 진행중입니다. (시도 {retry+1}/{max_retries})')
            
            if response.status_code != 200:
                print(f'오류: HTTP 상태 코드 {response.status_code}')
                if response.status_code in [403, 429]:  # 접근 거부 또는 너무 많은 요청
                    handle_blocking()
                    continue
                return None
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # 차단 여부 확인
            if check_if_blocked(soup):
                handle_blocking()
                continue
                
            return soup
            
        except Exception as e:
            print(f'페이지 {page}번 요청 중 오류 발생: {str(e)}')
            time.sleep(retry * 2 + 1)  # 재시도 간격 증가
    
    print(f"페이지 {page}번: 최대 재시도 횟수 초과")
    return None

def call_content(page, title_num, max_retries=2):
    """게시글 내용 불러오는 함수 (차단 감지 및 재시도 로직 포함)"""
    global is_blocked
    
    # 차단 상태 확인
    if is_blocked and handle_blocking():
        return None
    
    url = f'https://gall.dcinside.com/mgallery/board/view/?id=dfip&no={title_num}&page={page}'
    
    for retry in range(max_retries):
        try:
            # 무작위 대기 시간 (차단 방지)
            delay = wait_time + random.uniform(0, 0.3)
            time.sleep(delay)
            
            # 랜덤 헤더 사용
            current_headers = get_random_headers()
            response = requests.get(url, headers=current_headers, timeout=10)
            
            if response.status_code != 200:
                if response.status_code in [403, 429]:  # 접근 거부 또는 너무 많은 요청
                    handle_blocking()
                    continue
                return None
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # 차단 여부 확인
            if check_if_blocked(soup):
                handle_blocking()
                continue
                
            return soup
            
        except Exception as e:
            print(f'{page}페이지, {title_num}번 게시글: 요청 오류 {str(e)}')
            time.sleep(retry * 2 + 1)  # 재시도 간격 증가
    
    return None

def process_page(page_num):
    """한 페이지 처리하는 함수"""
    # 결과를 담을 리스트
    page_results = {
        'title_num': [],
        'write_time': [],
        'nickname': [],
        'title_text': [],
        'content': []
    }
    
    # 페이지 불러오기
    soup = call_site(page_num)
    if soup is None:
        return page_results, False
    
    try:
        # tbody 요소 확인
        tbody = soup.find('tbody')
        if tbody is None:
            print(f"페이지 {page_num}: tbody를 찾을 수 없음")
            return page_results, False
            
        # 게시글 목록 가져오기
        article_list = tbody.find_all('tr')
        
        # 페이지 내 게시글 처리
        for i in range(1, len(article_list)):
            article = article_list[i]
            
            # 게시글 번호
            try:
                num_td = article.find("td", {"class": "gall_num"})
                if num_td is None:
                    continue
                    
                title_num = num_td.text
                if not title_num.isdigit():  # 공지사항 건너뛰기
                    continue
            except:
                continue
            
            # 닉네임 추출
            try:
                nickname_span = article.find("span", {'class': "nickname in"})
                if nickname_span:
                    nickname = nickname_span.text
                else:
                    sub_nickname = article.find("span", {'class': "nickname"})['title']
                    sub_ip = article.find("span", {'class': "ip"}).text
                    nickname = f"{sub_nickname}{sub_ip}"
            except:
                nickname = "알 수 없음"
            
            # 게시글 제목
            try:
                title_links = article.find_all("a")
                title_text = title_links[0].text.strip() if title_links else "제목 없음"
            except:
                title_text = "제목 없음"
            
            # 작성 시간
            try:
                date_td = article.find("td", {'class': 'gall_date'})
                if date_td and 'title' in date_td.attrs:
                    write_time = date_td['title']
                    write_time = pd.to_datetime(write_time).strftime("%Y-%m-%d")
                else:
                    continue
            except:
                continue
            
            # 날짜 기준으로 중단 여부 결정
            if write_time < cut_off_date:
                return page_results, True  # 중단 신호 반환
            
            # 게시글 내용 가져오기
            try:
                content_soup = call_content(page=page_num, title_num=title_num)
                if content_soup is None:
                    continue
                    
                content_div = content_soup.find('div', class_='write_div')
                if content_div is None:
                    continue
                    
                content = content_div.text.strip().split()
                
                # 데이터 저장
                page_results['title_num'].append(title_num)
                page_results['title_text'].append(title_text)
                page_results['nickname'].append(nickname)
                page_results['write_time'].append(write_time)
                page_results['content'].append(content)
            except Exception as e:
                print(f"내용 추출 오류 (페이지 {page_num}, 게시글 {title_num}): {str(e)}")
                continue
    
    except Exception as e:
        print(f"페이지 {page_num} 처리 중 오류: {str(e)}")
    
    return page_results, False

def save_progress(results, last_page):
    """현재까지의 결과 저장"""
    if not results or all(len(results[key]) == 0 for key in results):
        print("저장할 데이터가 없습니다.")
        return
    
    try:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"dnf_dcinside_progress_{timestamp}_to_page_{last_page}.csv"
        
        df = pd.DataFrame(results)
        df.to_csv(filename, index=False)
        print(f"진행 상황 저장 완료: {filename} ({len(df)}개 게시글)")
    except Exception as e:
        print(f"저장 중 오류 발생: {str(e)}")

def main():
    # 데이터 축적용 딕셔너리
    all_results = {
        'title_num': [],
        'write_time': [],
        'nickname': [],
        'title_text': [],
        'content': []
    }
    
    # 마지막으로 처리한 페이지 번호
    last_processed_page = start_page - 1
    
    # 프로그램 시작 시간
    start_time = time.time()
    
    try:
        # 페이지 범위를 나누어 처리 (한 번에 너무 많은 페이지를 처리하지 않음)
        chunk_size = 20  # 차단 가능성 줄이기 위해 청크 크기 감소
        
        for chunk_start in range(start_page, end_page + 1, chunk_size):
            chunk_end = min(chunk_start + chunk_size - 1, end_page)
            print(f"페이지 {chunk_start}~{chunk_end} 처리 중...")
            
            # 차단 상태 확인 및 처리
            if is_blocked:
                while handle_blocking():
                    pass  # 차단 상태가 해제될 때까지 대기
            
            # 병렬 처리를 위한 ThreadPoolExecutor 사용
            with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
                # 각 페이지에 process_page 함수 실행
                futures = {executor.submit(process_page, page_num): page_num 
                          for page_num in range(chunk_start, chunk_end + 1)}
                
                # 작업 결과 수집
                should_break = False
                for future in concurrent.futures.as_completed(futures):
                    page_num = futures[future]
                    
                    try:
                        page_results, stop_signal = future.result()
                        
                        # 결과 합치기
                        for key in all_results:
                            all_results[key].extend(page_results[key])
                        
                        last_processed_page = max(last_processed_page, page_num)
                        print(f"페이지 {page_num} 완료: {len(page_results['title_num'])}개의 게시글 처리됨")
                        
                        # 중단 신호 처리
                        if stop_signal:
                            should_break = True
                            break
                            
                    except Exception as e:
                        print(f"페이지 {page_num} 처리 실패: {str(e)}")
            
            # 중간 결과 저장
            save_progress(all_results, last_processed_page)
            
            # 차단 상태 확인
            if is_blocked:
                print("차단이 감지되었습니다. 대기 후 다시 시작합니다.")
                while handle_blocking():
                    pass  # 차단 상태가 해제될 때까지 대기
            
            # 중단 신호 확인
            if should_break:
                print("날짜 기준 도달로 크롤링 종료")
                break
            
            # 청크 사이에 잠시 대기 (차단 방지)
            time.sleep(random.uniform(1, 3))
    
    except KeyboardInterrupt:
        print("\n사용자에 의해 중단됨")
    except Exception as e:
        print(f"예기치 못한 오류: {str(e)}")
    finally:
        # 최종 결과 저장
        final_df = pd.DataFrame(all_results)
        final_df.to_csv(f"dnf_dcinside_content_414-{last_processed_page}.csv", index=False)
        print(f"크롤링 완료! 총 {len(final_df)}개의 게시글 수집됨 (페이지 {start_page}~{last_processed_page})")
        
        # 소요 시간 출력
        elapsed_time = time.time() - start_time
        print(f"총 소요 시간: {elapsed_time:.2f}초")

if __name__ == "__main__":
    main()

페이지 3180~3199 처리 중...
페이지 3181번 진행중입니다. (시도 1/3)
차단이 감지되었습니다. 120초 대기합니다.
페이지 3180번 진행중입니다. (시도 1/3)
차단 상태입니다. 120초 더 대기 중...
페이지 3180번 진행중입니다. (시도 2/3)
차단 상태입니다. 109초 더 대기 중...
페이지 3181번 진행중입니다. (시도 2/3)
차단 상태입니다. 109초 더 대기 중...
페이지 3181번 진행중입니다. (시도 3/3)
차단 상태입니다. 98초 더 대기 중...
페이지 3180번 진행중입니다. (시도 3/3)
차단 상태입니다. 97초 더 대기 중...
페이지 3181번: 최대 재시도 횟수 초과
차단 상태입니다. 88초 더 대기 중...
페이지 3181 완료: 0개의 게시글 처리됨
페이지 3180번: 최대 재시도 횟수 초과
차단 상태입니다. 87초 더 대기 중...
페이지 3180 완료: 0개의 게시글 처리됨
차단 상태입니다. 78초 더 대기 중...페이지 3182 완료: 0개의 게시글 처리됨

차단 상태입니다. 77초 더 대기 중...
페이지 3183 완료: 0개의 게시글 처리됨
차단 상태입니다. 68초 더 대기 중...페이지 3184 완료: 0개의 게시글 처리됨

차단 상태입니다. 67초 더 대기 중...
페이지 3185 완료: 0개의 게시글 처리됨
차단 상태입니다. 58초 더 대기 중...페이지 3186 완료: 0개의 게시글 처리됨

차단 상태입니다. 57초 더 대기 중...
페이지 3187 완료: 0개의 게시글 처리됨
차단 상태입니다. 48초 더 대기 중...
페이지 3188 완료: 0개의 게시글 처리됨
차단 상태입니다. 47초 더 대기 중...
페이지 3189 완료: 0개의 게시글 처리됨
차단 상태입니다. 38초 더 대기 중...
차단 상태입니다. 37초 더 대기 중...
차단 상태입니다. 28초 더 대기 중...
차단 상태입니다. 27초 더 대기 중...
차단 상태입니다. 18초 더 대기 중...
차단 상태입니다. 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import concurrent.futures
import datetime

# 기본 설정
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}
start_page = 3700
end_page = 21650
cut_off_date = '2024-12-01'
max_workers = 3  # 동시에 실행할 작업 수
PAUSE_TIME = 300  # 차단 감지 시 대기 시간(초) - 5분
MAX_ERRORS_ALLOWED = 3  # 허용 가능한 최대 오류 수

# 여러 개의 User-Agent를 사용하여 차단 방지
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0'
]

# 오류 카운터
error_counter = 0

def get_random_headers():
    """랜덤 User-Agent를 포함한 헤더 반환"""
    return {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/html,application/xhtml+xml,application/xml',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    }

def check_if_blocked(soup):
    """차단 여부 확인 함수"""
    # 디시인사이드의 차단 페이지 특성을 확인
    # 1. tbody가 없는 경우
    if soup is None or soup.find('tbody') is None:
        return True
    
    # 2. 특정 오류 메시지가 포함된 경우
    error_texts = ['접근이 차단되었습니다', '비정상적인 접근', '일시적으로 접근이 제한되었습니다']
    page_text = soup.get_text().lower()
    for error in error_texts:
        if error.lower() in page_text:
            return True
    
    return False

def pause_crawler():
    """크롤러를 5분간 일시 정지하는 함수"""
    print(f"\n차단이 감지되었습니다. {PAUSE_TIME}초({PAUSE_TIME/60:.1f}분) 동안 모든 작업을 중단합니다.")
    print(f"현재 시간: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # 30초마다 남은 시간 표시하며 대기
    for remaining in range(PAUSE_TIME, 0, -30):
        print(f"남은 대기 시간: {remaining}초({remaining/60:.1f}분)")
        time.sleep(min(30, remaining))
    
    print(f"대기 완료. 크롤링을 재개합니다. 시간: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

def should_pause_due_to_errors():
    """오류 개수가 허용치를 초과하는지 확인"""
    global error_counter
    if error_counter > MAX_ERRORS_ALLOWED:
        print(f"연속 오류 개수({error_counter}개)가 허용치({MAX_ERRORS_ALLOWED}개)를 초과했습니다.")
        error_counter = 0  # 오류 카운터 초기화
        return True
    return False

def reset_error_counter():
    """오류 카운터 초기화"""
    global error_counter
    error_counter = 0

def call_site(page):
    """페이지 불러오는 함수"""
    global error_counter
    
    url = f'https://gall.dcinside.com/mgallery/board/lists/?id=dfip&page={page}'
    
    try:
        # 무작위 대기 시간 (차단 방지)
        wait_time = random.uniform(0.2, 1.0)
        time.sleep(wait_time)
        
        # 랜덤 헤더 사용
        current_headers = get_random_headers()
        response = requests.get(url, headers=current_headers, timeout=10)
        
        print(f'페이지 {page}번 진행중입니다. (대기시간: {wait_time:.2f}초)')
        
        if response.status_code != 200:
            print(f'오류: HTTP 상태 코드 {response.status_code}')
            if response.status_code in [403, 429]:  # 접근 거부 또는 너무 많은 요청
                error_counter += 1
                if should_pause_due_to_errors():
                    return None, True  # 차단 신호 반환
                return None, False
            
            error_counter += 1
            return None, False
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 차단 여부 확인
        if check_if_blocked(soup):
            error_counter += 1
            if should_pause_due_to_errors():
                return None, True  # 차단 신호 반환
            return None, False
        
        # 성공했으므로 오류 카운터 초기화
        reset_error_counter()
        return soup, False
        
    except Exception as e:
        print(f'페이지 {page}번 요청 중 오류 발생: {str(e)}')
        error_counter += 1
        if should_pause_due_to_errors():
            return None, True
        return None, False

def call_content(page, title_num):
    """게시글 내용 불러오는 함수"""
    global error_counter
    
    url = f'https://gall.dcinside.com/mgallery/board/view/?id=dfip&no={title_num}&page={page}'
    
    try:
        # 무작위 대기 시간 (차단 방지)
        wait_time = random.uniform(0.2, 1.0)
        time.sleep(wait_time)
        
        # 랜덤 헤더 사용
        current_headers = get_random_headers()
        response = requests.get(url, headers=current_headers, timeout=10)
        
        if response.status_code != 200:
            if response.status_code in [403, 429]:  # 접근 거부 또는 너무 많은 요청
                error_counter += 1
                if should_pause_due_to_errors():
                    return None, True  # 차단 신호 반환
                return None, False
            
            error_counter += 1
            return None, False
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 차단 여부 확인
        if check_if_blocked(soup):
            error_counter += 1
            if should_pause_due_to_errors():
                return None, True  # 차단 신호 반환
            return None, False
            
        # 성공했으므로 오류 카운터 초기화
        reset_error_counter()
        return soup, False
        
    except Exception as e:
        print(f'{page}페이지, {title_num}번 게시글: 요청 오류 {str(e)}')
        error_counter += 1
        if should_pause_due_to_errors():
            return None, True
        return None, False

def process_page(page_num):
    """한 페이지 처리하는 함수"""
    # 결과를 담을 리스트
    page_results = {
        'title_num': [],
        'write_time': [],
        'nickname': [],
        'title_text': [],
        'content': []
    }
    
    # 페이지 불러오기
    soup, is_blocked = call_site(page_num)
    if is_blocked:
        return page_results, False, True  # 차단 신호 반환
    
    if soup is None:
        return page_results, False, False
    
    try:
        # tbody 요소 확인
        tbody = soup.find('tbody')
        if tbody is None:
            print(f"페이지 {page_num}: tbody를 찾을 수 없음")
            return page_results, False, False
            
        # 게시글 목록 가져오기
        article_list = tbody.find_all('tr')
        
        # 페이지 내 게시글 처리
        for i in range(1, len(article_list)):
            article = article_list[i]
            
            # 게시글 번호
            try:
                num_td = article.find("td", {"class": "gall_num"})
                if num_td is None:
                    continue
                    
                title_num = num_td.text
                if not title_num.isdigit():  # 공지사항 건너뛰기
                    continue
            except:
                continue
            
            # 닉네임 추출
            try:
                nickname_span = article.find("span", {'class': "nickname in"})
                if nickname_span:
                    nickname = nickname_span.text
                else:
                    sub_nickname = article.find("span", {'class': "nickname"})['title']
                    sub_ip = article.find("span", {'class': "ip"}).text
                    nickname = f"{sub_nickname}{sub_ip}"
            except:
                nickname = "알 수 없음"
            
            # 게시글 제목
            try:
                title_links = article.find_all("a")
                title_text = title_links[0].text.strip() if title_links else "제목 없음"
            except:
                title_text = "제목 없음"
            
            # 작성 시간
            try:
                date_td = article.find("td", {'class': 'gall_date'})
                if date_td and 'title' in date_td.attrs:
                    write_time = date_td['title']
                    write_time = pd.to_datetime(write_time).strftime("%Y-%m-%d")
                else:
                    continue
            except:
                continue
            
            # 날짜 기준으로 중단 여부 결정
            if write_time < cut_off_date:
                return page_results, True, False  # 중단 신호 반환 (날짜 기준)
            
            # 게시글 내용 가져오기
            try:
                content_soup, content_blocked = call_content(page=page_num, title_num=title_num)
                if content_blocked:
                    return page_results, False, True  # 차단 신호 반환
                
                if content_soup is None:
                    continue
                    
                content_div = content_soup.find('div', class_='write_div')
                if content_div is None:
                    continue
                    
                content = content_div.text.strip().split()
                
                # 데이터 저장
                page_results['title_num'].append(title_num)
                page_results['title_text'].append(title_text)
                page_results['nickname'].append(nickname)
                page_results['write_time'].append(write_time)
                page_results['content'].append(content)
            except Exception as e:
                print(f"내용 추출 오류 (페이지 {page_num}, 게시글 {title_num}): {str(e)}")
                continue
    
    except Exception as e:
        print(f"페이지 {page_num} 처리 중 오류: {str(e)}")
    
    return page_results, False, False  # (결과, 날짜 중단 신호, 차단 신호)

def save_progress(results, last_page):
    """현재까지의 결과 저장"""
    if not results or all(len(results[key]) == 0 for key in results):
        print("저장할 데이터가 없습니다.")
        return
    
    try:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"dnf_dcinside_progress_{timestamp}_to_page_{last_page}.csv"
        
        df = pd.DataFrame(results)
        df.to_csv(filename, index=False)
        print(f"진행 상황 저장 완료: {filename} ({len(df)}개 게시글)")
    except Exception as e:
        print(f"저장 중 오류 발생: {str(e)}")

def main():
    # 데이터 축적용 딕셔너리
    all_results = {
        'title_num': [],
        'write_time': [],
        'nickname': [],
        'title_text': [],
        'content': []
    }
    
    # 마지막으로 처리한 페이지 번호
    last_processed_page = start_page - 1
    
    # 프로그램 시작 시간
    start_time = time.time()
    
    try:
        # 페이지 범위를 나누어 처리 (한 번에 너무 많은 페이지를 처리하지 않음)
        chunk_size = 20  # 차단 가능성 줄이기 위해 청크 크기 감소
        
        current_page = start_page
        while current_page <= end_page:
            chunk_end = min(current_page + chunk_size - 1, end_page)
            print(f"페이지 {current_page}~{chunk_end} 처리 중...")
            
            # 차단 감지 플래그
            blocking_detected = False
            date_cutoff_reached = False
            
            # 병렬 처리를 위한 ThreadPoolExecutor 사용
            with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
                # 각 페이지에 process_page 함수 실행
                futures = {executor.submit(process_page, page_num): page_num 
                          for page_num in range(current_page, chunk_end + 1)}
                
                # 작업 결과 수집
                for future in concurrent.futures.as_completed(futures):
                    page_num = futures[future]
                    
                    try:
                        page_results, date_signal, block_signal = future.result()
                        
                        # 차단 감지 확인
                        if block_signal:
                            print(f"페이지 {page_num}에서 차단이 감지되었습니다.")
                            blocking_detected = True
                            break
                        
                        # 결과 합치기
                        for key in all_results:
                            all_results[key].extend(page_results[key])
                        
                        if len(page_results['title_num']) > 0:  # 성공적으로 데이터를 가져온 경우만 업데이트
                            last_processed_page = max(last_processed_page, page_num)
                        
                        print(f"페이지 {page_num} 완료: {len(page_results['title_num'])}개의 게시글 처리됨")
                        
                        # 날짜 기준 신호 처리
                        if date_signal:
                            print(f"페이지 {page_num}에서 날짜 기준({cut_off_date}) 이전 게시글 발견")
                            date_cutoff_reached = True
                            break
                            
                    except Exception as e:
                        print(f"페이지 {page_num} 처리 실패: {str(e)}")
            
            # 작업 중단 여부 처리
            if blocking_detected:
                # 중간 결과 저장
                save_progress(all_results, last_processed_page)
                
                # 5분간 대기
                pause_crawler()
                
                # 오류 카운터 초기화
                reset_error_counter()
                
                # 현재 페이지부터 다시 시작 (실패한 페이지부터)
                current_page = last_processed_page + 1
            
            elif date_cutoff_reached:
                print("날짜 기준 도달로 크롤링 종료")
                break
            
            else:
                # 중간 결과 저장
                save_progress(all_results, last_processed_page)
                
                # 다음 청크로 진행
                current_page = chunk_end + 1
                
                # 청크 사이에 잠시 대기 (차단 방지)
                time.sleep(random.uniform(1, 3))
    
    except KeyboardInterrupt:
        print("\n사용자에 의해 중단됨")
    except Exception as e:
        print(f"예기치 못한 오류: {str(e)}")
    finally:
        # 최종 결과 저장
        final_df = pd.DataFrame(all_results)
        final_df.to_csv(f"dnf_dcinside_content_414-{last_processed_page}.csv", index=False)
        print(f"크롤링 완료! 총 {len(final_df)}개의 게시글 수집됨 (페이지 {start_page}~{last_processed_page})")
        
        # 소요 시간 출력
        elapsed_time = time.time() - start_time
        hours, remainder = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"총 소요 시간: {int(hours)}시간 {int(minutes)}분 {seconds:.2f}초")

if __name__ == "__main__":
    main()

페이지 3700~3719 처리 중...
페이지 3702번 진행중입니다. (대기시간: 0.53초)
페이지 3701번 진행중입니다. (대기시간: 0.71초)
페이지 3700번 진행중입니다. (대기시간: 0.73초)
페이지 3701 완료: 50개의 게시글 처리됨
페이지 3703번 진행중입니다. (대기시간: 0.66초)
페이지 3700 완료: 50개의 게시글 처리됨
페이지 3704번 진행중입니다. (대기시간: 0.45초)
페이지 3702 완료: 50개의 게시글 처리됨
페이지 3705번 진행중입니다. (대기시간: 0.59초)
페이지 3703 완료: 50개의 게시글 처리됨
페이지 3706번 진행중입니다. (대기시간: 0.90초)
페이지 3704 완료: 49개의 게시글 처리됨
페이지 3707번 진행중입니다. (대기시간: 0.83초)
페이지 3707 완료: 0개의 게시글 처리됨
페이지 3708번 진행중입니다. (대기시간: 0.92초)
페이지 3705 완료: 50개의 게시글 처리됨
페이지 3709번 진행중입니다. (대기시간: 0.69초)
페이지 3708 완료: 37개의 게시글 처리됨
페이지 3710번 진행중입니다. (대기시간: 0.45초)
페이지 3706 완료: 50개의 게시글 처리됨
페이지 3711번 진행중입니다. (대기시간: 0.41초)
페이지 3709 완료: 50개의 게시글 처리됨
페이지 3712번 진행중입니다. (대기시간: 0.30초)
페이지 3710 완료: 50개의 게시글 처리됨
페이지 3713번 진행중입니다. (대기시간: 0.65초)
페이지 3711 완료: 50개의 게시글 처리됨
페이지 3714번 진행중입니다. (대기시간: 0.93초)
페이지 3712 완료: 50개의 게시글 처리됨
페이지 3715번 진행중입니다. (대기시간: 0.32초)
페이지 3713 완료: 50개의 게시글 처리됨
페이지 3716번 진행중입니다. (대기시간: 0.84초)
페이지 3714 완료: 50개의 게시글 처리됨
페이지 3717번 진행중입니다. (대기시간: 0.87초)
페이지 3715 완료: 

In [ ]:
#받고싶은 페이지 입력
start_page = 3150
page = 21650
wait_time = 0.2 # 기다리는 시간 2초

# 데이터 축적
title_lst = []
nickname_lst = []
time_lst = []
num_lst = []
content_lst = []

should_break = False

# try :
for i in range(1805, page + 1) :
    
    # 대기
    time.sleep(wait_time)

    # url 호출
    soup = call_site(i)
    
    if soup == None :
        break

    # 데이터 로드
    article_list = soup.find('tbody').find_all('tr')

    # 게시글 필터링 
    for i in range(1, len(article_list)) : 
        # html 전처리
        article = article_list[i]

        # 게시글 번호
        title_num = article.find("td", {"class" : "gall_num"}).text # 게시글 번호
        if not title_num.isdigit() : # 없으면 넘기기
            continue

        # 닉네임 추출
        ## 디시인사이드 특성상 'ㅇㅇ'이라는 닉이 많아서 고정닉이랑 ㅇㅇ이랑 처리가 필요함 = 따로 예외처리 필요
        try :
            nickname = article.find("span", {'class' : "nickname in"}).text
        except AttributeError:
            sub_nickname = article.find("span", {'class' : "nickname"})['title']
            sub_ip = article.find("span", {'class' : "ip"}).text
            nickname = f"{sub_nickname}{sub_ip}"

        # 게시글 제목
        title_text = article.find_all("a")[0].text.strip()

        # 게시글 작성 시간 추출
        ## 던파 공홈과는 달리 따로 gall_date라고 정해져 있는게 있어서 처리는 필요 없을듯
        write_time = article.find("td", {'class' : 'gall_date'})['title']
        write_time = pd.to_datetime(write_time).strftime("%Y-%m-%d")

        if write_time < '2024-12-01' :
            should_break = True
            break

        # 게시글 내용 추출
        ## 오류시 건너뛰기
        try :
            soup = call_content(page = i, title_num = title_num)
        except :
            continue

        try :
            content = soup.find('div', class_ = 'write_div').text.strip().split()
        except :
            continue

        # 리스트에 데이터 축적
        num_lst.append(title_num)
        title_lst.append(title_text)
        nickname_lst.append(nickname)
        time_lst.append(write_time)
        content_lst.append(content)

    if should_break :
        print("크롤링 끝")
        break

print("DC_content_Done")

# 데이터 프레임 축적
dc_df = pd.DataFrame({
    'title_num' : num_lst,
    'write_time' : time_lst,
    'nickname' : nickname_lst,
    'title_text' : title_lst,
    'content' : content_lst
})
dc_df.to_csv(f"dnf_dcinside_content_414-{page}.csv")

# except Exception as e:
#     dc_df = pd.DataFrame({
#     'title_num' : num_lst,
#     'write_time' : time_lst,
#     'nickname' : nickname_lst,
#     'title_text' : title_lst,
#     'content' : content_lst})
#     dc_df.to_csv("dnf_dcinside_content_error.csv")
    
#     print('예기치 못한 오류 발생')
#     print(e)

페이지 1805번 진행중입니다.
23페이지, 1092103번 게시글 : error
페이지 1806번 진행중입니다.


In [ ]:
# 데이터 프레임 축적
dc_df = pd.DataFrame({
    'title_num' : num_lst,
    'write_time' : time_lst,
    'nickname' : nickname_lst,
    'title_text' : title_lst,
    'content' : content_lst
})
dc_df.to_csv("dnf_dcinside_content_1805-.csv")

In [1]:
dc_df.info()

NameError: name 'dc_df' is not defined

In [41]:
dc_df.drop_duplicates(subset = 'title_text')

,title_num,write_time,nickname,title_text,content
0,1159973,2025-04-09,ㅇㅇ(59.5),12증버퍼는 대가리 깨져도 무조건 황금향임?,"[황금향, 제외, 타세트, 악세3태초여도, 황금향이, 뺨아리, 줘팬다던디, ㄹㅇ임?]"
1,1159972,2025-04-09,ㅇㅇ,채팅제한 장난하니? 동업자 정신이 없노,"[엘리브가, 경매물품, 1개, 주면, 욱할수도, 있지지도, 화났으면서, 왜, 신고함?]"
2,1159971,2025-04-09,RaTe,아 20 90 올실패는 씨발종민아,"[아, 제발….왜그ㅐ라나한테]"
3,1159970,2025-04-09,ㅇㅇ(121.182),유독 파밍이 더딘 캐릭을 어찌해야하나,"[같은, 계시를, 먹였는데, 먹어오는게업네딜러면, 유기하는데, 버퍼라ㅅㅂ...]"
4,1159969,2025-04-09,ㅇㅇ(14.53),레이드 광적으로 싫어하는 애들이 많네,"[싱글, 보상, 이야기, 많은, 것, 보면]"
...,...,...,...,...,...
27206,1132634,2025-04-06,키탁,패황vs용독에서,"[결국, 용독이, 1군으로, 올라감, ㅋㅋㅋㅋㅋㅋ, ㅋㅋㅋㅋㅋㅋ, 아, 패황, 1군..."
27207,1132633,2025-04-06,ㅇㅇ(123.248),보통 팔목반 제외 에픽초월은 몇부위 남았을때 함?,"[지금, 무기, 에거시고태초1부위, 비고, 에픽, 3부위, 비는데그냥, 에픽3부위,..."
27208,1132632,2025-04-06,ㅇㅇ,본캐 노태초 황금향에픽풀셋인데 헬계속돌려야함?,"[무기도, 에거시임..]"
27215,1132631,2025-04-06,ㅇㅇ(116.33),요새 배럭 환요는 돌아줘야하냐?,"[한, 캐릭당, 2분, 걸리니까날이, 가면, 갈, 수록진짜, 하기, 싫어지던데..]"
